<br><br><p style="text-align:center;font-size:2em;font-weight: bold;text-decoration: underline">Data Clean</p>

<p>This notebook serves as cleaning code for the data. 
At the end all the original csv files have the following columns:<br><br>
[Start date, Start station, End station, Holiday, Business Day, <br>Start time, Same Location, Duration, TimetoDest, Member Type]</p>
<p> Holiday, Business Day, and Same location (Start and End Stations are the same) are indicator variables 1 for true, 0 for false.</p>
<p> The Duration is the duration of time the bike was used and the TimetoDest is Google's prediction for how long it takes to bike between the start and end station.</p>

In [ ]:
# Load in the Libraries
import numpy as np
import pandas as pd
import re
import datetime

In [ ]:
#### Federal Holidays ####
DCHolidays=[ 
    datetime.date(2017, 1, 2), # New Years Holiday (in lieu)
    datetime.date(2017, 1, 16), # MLK Day
    datetime.date(2017, 1, 20), # Inauguration Day
    datetime.date(2017, 2, 20), # Presidents' Day
    datetime.date(2017, 4, 17), # Emancipation Day
    datetime.date(2017, 5, 29), # Memorial Day
    datetime.date(2017, 7, 4), # Independence Day
    datetime.date(2017, 9, 4), # Labor Day
    datetime.date(2017, 10, 9), # Columbus Day
    datetime.date(2017, 11, 23), # Thanksgiving
    datetime.date(2017, 12, 25), # Christmas
    datetime.date(2016, 1, 1), # New Years
    datetime.date(2016, 1, 18), # MLK Day
    datetime.date(2016, 2, 15), # Presidents' Day
    datetime.date(2016, 4, 15), # Emancipation Day
    datetime.date(2016, 5, 30), # Memorial Day
    datetime.date(2016, 7, 4), # Independence Day
    datetime.date(2016, 9, 5), # Labor Day
    datetime.date(2016, 10, 10), # Columbus Day
    datetime.date(2016, 11, 11), # Veterans Day
    datetime.date(2016, 11, 24), # Thanksgiving
    datetime.date(2016, 12, 26), # Christmas (in lieu)
    datetime.date(2015, 1, 1), # New Years
    datetime.date(2015, 1, 19), # MLK Day
    datetime.date(2015, 4, 16), # Emancipation Day
    datetime.date(2015, 5, 25), # Memorial Day
    datetime.date(2015, 7, 3), # Independence Day (observed)
    datetime.date(2015, 9, 7), # Labor Day
    datetime.date(2015, 10, 12), # Columbus Day
    datetime.date(2015, 11, 11), # Veterans Day
    datetime.date(2015, 11, 26), # Thanksgiving
    datetime.date(2015, 12, 25), # Christmas
    datetime.date(2014, 1, 1), # New Years 
    datetime.date(2014, 1, 20), # MLK Day
    datetime.date(2014, 2, 17), # Presidents' Day
    datetime.date(2014, 4, 16), # Emancipation Day
    datetime.date(2014, 5, 26), # Memorial Day
    datetime.date(2014, 7, 4), # Independence Day
    datetime.date(2014, 9, 1), # Labor Day
    datetime.date(2014, 10, 13), # Columbus Day
    datetime.date(2014, 11, 11), # Veterans Day
    datetime.date(2014, 11, 27), # Thanksgiving
    datetime.date(2014, 12, 25), # Christmas
    datetime.date(2014, 12, 26), # Christmas (Day after)
    datetime.date(2013, 1, 1), # New Years
    datetime.date(2013, 1, 21), # Inauguration Day, MLK
    datetime.date(2013, 2, 18), # Presidents' Day
    datetime.date(2013, 4, 16), # Emancipation Day
    datetime.date(2013, 5, 27), # Memorial Day
    datetime.date(2013, 7, 4), # Independence Day
    datetime.date(2013, 9, 2), # Labor Day
    datetime.date(2013, 10, 14), # Columbus Day
    datetime.date(2013, 11, 11), # Veterans Day
    datetime.date(2013, 11, 28), # Thanksgiving
    datetime.date(2013, 12, 25), # Christmas
    datetime.date(2012, 1, 2), # New Years Holiday (in lieu)
    datetime.date(2012, 1, 16), # MLK Day
    datetime.date(2012, 2, 20), # Presidents' Day
    datetime.date(2012, 4, 16), # Emancipation Day
    datetime.date(2012, 5, 28), # Memorial Day
    datetime.date(2012, 7, 4), # Independence Day
    datetime.date(2012, 9, 4), # Labor Day
    datetime.date(2012, 10, 8), # Columbus Day
    datetime.date(2012, 11, 12), # Veterans Day
    datetime.date(2012, 11, 22), # Thanksgiving
    datetime.date(2012, 12, 25), # Christmas
    datetime.date(2011, 1, 17), # MLK Day
    datetime.date(2011, 2, 21), # Presidents' Day
    datetime.date(2011, 4, 15), # Emancipation Day
    datetime.date(2011, 5, 30), # Memorial Day
    datetime.date(2011, 7, 4), # Independence Day
    datetime.date(2011, 9, 5), # Labor Day
    datetime.date(2011, 10, 10), # Columbus Day
    datetime.date(2011, 11, 11), # Veterans Day
    datetime.date(2011, 11, 24), # Thanksgiving
    datetime.date(2011, 12, 26), # Christmas (in lieu)
    datetime.date(2010, 9, 6), # Labor Day
    datetime.date(2010, 10, 11), # Columbus Day
    datetime.date(2010, 11, 11), # Veterans Day
    datetime.date(2010, 11, 25), # Thanksgiving
    datetime.date(2010, 12, 24), # Christmas (in lieu)
    datetime.date(2010, 12, 31) # New Years Holiday (in lieu)
]

In [ ]:
# Reads in a string and converts it to seconds
def TimetoSec(timestring):
    result=re.split("\D+", timestring)
    hour=int(result[0])
    minute=int(result[1])
    second=int(result[2])
    return hour*3600+minute*60+second

In [ ]:
# Grab the address so that we can looking it up in our address data csv file
def StripAddress(addr):
    result=re.search('\d\d\d\d\d',addr)
    if(result is None):
        return addr
    else:
        return addr[0:result.span()[0]-2]

In [ ]:
# Determines if it's a business day, I.E. a weekday and not a federal holiday
def isBusinessDay(Date1):
    return int((Date1 not in DCHolidays) and datetime.date.weekday(Date1)<5)

In [ ]:
# Finds the Station Index in the address data file
def findIndex(station):
    try:
        
        # These two locations have no address so we return 0 and later we will change the Time to Destination to 0
        if(station=='Alta Bicycle Share Demonstration Station' or station=='Birthday Station'):
            return 0
        else:
            return addressdata.index[addressdata['Address']==station][0]
    except:
        # If there is a station that doesn't match output it
        print(station)

In [ ]:
# Some data the station name was incorrectly labelled so we convert to the true string
def FixStation(station):
    if(station=='Virginia Square'):
        return 'Virginia Square Metro / N Monroe St & 9th St N'
    elif(station=='Central Library'):
        return 'Central Library / N Quincy St & 10th St N'
    else:
         return station

In [ ]:
# The address Data file
addressdata=pd.read_csv('Myaddrdata.csv')

In [ ]:
# The Google calculated bicycle distance between points
distancearray=np.load('distancearray.npy')

In [ ]:
# Change this to whichever data set you are to be cleaned (Be sure to change the IF statement below also)
DatabaseString='HistoryData/2016-Q4-Trips-History-Data.csv'
data=pd.read_csv(DatabaseString)

# 2017 Data reverts back to the Duration column BUT now it's in miliseconds as before it was in a different format
if(DatabaseString=='HistoryData/2017-Q1-Trips-History-Data.csv'):
    data.rename(columns={'Duration': 'Total duration (ms)'},inplace=True)

In [ ]:
# The Files have different names for the data and this piece of code uniformizes each dataframe
if('Subscription Type' in list(data)):
    data.rename(columns={'Subscription Type': 'Member Type'},inplace=True)
if('Subscription type' in list(data)):
    data.rename(columns={'Subscription type': 'Member Type'},inplace=True)
if('Member type' in list(data)):
    data.rename(columns={'Member type': 'Member Type'},inplace=True)
if('Subscriber Type' in list(data)):
    data.rename(columns={'Subscriber Type': 'Member Type'},inplace=True)
if('Bike Key' in list(data)):
    data.rename(columns={'Bike Key': 'Member Type'},inplace=True)
if('Type' in list(data)):
    data.rename(columns={'Type': 'Member Type'}, inplace=True)
if('Account type' in list(data)):
    data.rename(columns={'Account type': 'Member Type'}, inplace=True)
if('Start Station' in list(data)):
    data.rename(columns={'Start Station': 'Start station'},inplace=True)
if('End Station' in list(data)):
    data.rename(columns={'End Station': 'End station'},inplace=True)
if('Start time' in list(data)):
    data.rename(columns={'Start time': 'Start date'},inplace=True)
if('Bike number' in list(data)):
    data.rename(columns={'Bike number': 'Bike#'},inplace=True)
if('Bike #' in list(data)):
    data.rename(columns={'Bike #': 'Bike#'},inplace=True)
if('Duration (ms)' in list(data)):
    data.rename(columns={'Duration (ms)': 'Total duration (ms)'},inplace=True)

In [ ]:
# Some files the data comes in hours/minutes/seconds, other files it comes in miliseconds
# This converts the files in either form to seconds
if('Total duration (ms)' in list(data)):
    data.rename(columns={'Total duration (ms)': 'Duration'},inplace=True)
    data['Duration']=data['Duration']//1000
else:
    data['Duration']=data['Duration'].apply(TimetoSec)

# Throw away incomplete data
data = data.dropna(subset=['Duration','Start date','Start station','End station','Member Type'])    

# Convert to datetime object (Depending on the file this has different inputs)
#data['Start date']=data['Start date'].apply(lambda x:datetime.datetime.strptime(x,'%Y-%m-%d %H:%M'))
data['Start date']=data['Start date'].apply(lambda x:datetime.datetime.strptime(x,'%m/%d/%Y %H:%M'))

# Strip off the Station Location
data['Start station']=data['Start station'].apply(StripAddress)
data['End station']=data['End station'].apply(StripAddress)

# 1 For Registered, 0 For Casual
data['Member Type']=data['Member Type'].apply(lambda x: int((x=='Registered'))) 

# 1 if a Federal holiday
data['Holiday']=data['Start date'].apply(lambda x: int(datetime.datetime.date(x) in DCHolidays))

# 1 if it's a Business Day
data['Business Day']=data['Start date'].apply(lambda x: isBusinessDay(datetime.datetime.date(x)))

# Drop Bike# and End Date
data.drop('Bike#',axis=1,inplace=True)
data.drop('End date',axis=1,inplace=True)

# Drop Station Numbers if they are in the data file
if('Start station number' in list(data)):
    data.drop('Start station number',axis=1,inplace=True)
if('End station number' in list(data)):
    data.drop('End station number',axis=1,inplace=True)

# Start Time
data['Start time']=data['Start date'].apply(lambda x: x.hour*60+x.minute)

# This line removes a chain warning that will pop up by using the following lines of code
# pd.options.mode.chained_assignment = None

# This is 1 if the bike starts and ends at the same location 0 otherwise
data['Same Location']=0
data['Same Location'][data['Start station']==data['End station']]=1

# Fix bad Station Names
data['Start station']=data['Start station'].apply(FixStation)
data['End station']=data['End station'].apply(FixStation)

In [ ]:
# This finds the tuples for looking at the distance array matrix for each row
# This code can take around 5-10 minutes per dataset
distanceindices=list(zip(list(data['Start station'].transform(findIndex)),
    list(data['End station'].transform(findIndex))))


In [ ]:
# Put in the Time to Destination from the distance array
data['TimetoDest']=list(map(lambda x: distancearray[x],distanceindices))

# The locations of these two stations are unknown so we just put the time to destination as 0
for r in data.index[data['Start station']=='Alta Bicycle Share Demonstration Station'].tolist():
    data['TimetoDest'][r]=0
for r in data.index[data['End station']=='Alta Bicycle Share Demonstration Station'].tolist():
    data['TimetoDest'][r]=0
for r in data.index[data['Start station']=='Birthday Station'].tolist():
    data['TimetoDest'][r]=0
for r in data.index[data['End station']=='Birthday Station'].tolist():
    data['TimetoDest'][r]=0
        

In [ ]:
# Reorder the data frame columns
data=data[['Start date', 'Start station', 'End station','Holiday', 'Business Day', 'Start time', 
      'Same Location','Duration','TimetoDest','Member Type']]

In [ ]:
# Write the information to a file
data.to_csv("DataCleaned164.csv",index=False)